***8 практика***

In [1]:
#импортируем библиотеки
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np

from emnist import extract_training_samples, extract_test_samples

In [2]:
#загружаем тренировочную и тестовые выборки
train_images, train_labels = extract_training_samples('letters')
test_images, test_labels = extract_test_samples('letters')


In [3]:
train_images = train_images / 255.0
test_images = test_images / 255.0

In [4]:
#преобразуем метки в вектор с единицей на правильном ответе
from keras.utils import to_categorical
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

In [5]:
#задаем архитектуру сети №1
from tensorflow.keras.layers import Dense, Activation, Flatten
from tensorflow.keras.models import Sequential
model = Sequential()
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(27, activation='softmax'))

Создаем колбэк тензорборда. Путь должен быть полным и не иметь русских букв

In [6]:
from keras.callbacks import TensorBoard

# Создание экземпляра TensorBoard
tensorboard_callback = TensorBoard(log_dir='C:\\Users\\chechkysh\\Desktop\\learn\\last\\piis\\5lab\\tblogs', histogram_freq=1)


In [8]:
model.compile(optimizer='adam',loss='categorical_crossentropy', metrics=['accuracy'])

После обучения в терминале вводим команду 

tensorboard --logdir=C:\\Users\\chechkysh\\Desktop\\learn\\last\\piis\\5lab\\tblogs

А потом заходим в браузере на http://localhost:6006/

In [9]:
model.fit(train_images, train_labels, validation_data=(test_images, test_labels), epochs=5, batch_size=128, callbacks=[tensorboard_callback])

Epoch 1/5


975/975 [==============================] - 8s 7ms/step - loss: 0.8047 - accuracy: 0.7688 - val_loss: 0.5112 - val_accuracy: 0.8511
Epoch 2/5
975/975 [==============================] - 7s 7ms/step - loss: 0.4297 - accuracy: 0.8701 - val_loss: 0.4080 - val_accuracy: 0.8758
Epoch 3/5
975/975 [==============================] - 6s 6ms/step - loss: 0.3445 - accuracy: 0.8937 - val_loss: 0.3656 - val_accuracy: 0.8890
Epoch 4/5
975/975 [==============================] - 6s 6ms/step - loss: 0.3008 - accuracy: 0.9051 - val_loss: 0.3567 - val_accuracy: 0.8878
Epoch 5/5
975/975 [==============================] - 6s 6ms/step - loss: 0.2714 - accuracy: 0.9138 - val_loss: 0.3306 - val_accuracy: 0.8966


In [ ]:
from scipy.ndimage import center_of_mass
import math
import cv2
import numpy as np

def getBestShift(img):
    cy,cx = center_of_mass(img)

    rows,cols = img.shape
    shiftx = np.round(cols/2.0-cx).astype(int)
    shifty = np.round(rows/2.0-cy).astype(int)

    return shiftx,shifty

def shift(img,sx,sy):
    rows,cols = img.shape
    M = np.float32([[1,0,sx],[0,1,sy]])
    shifted = cv2.warpAffine(img,M,(cols,rows))
    return shifted

def rec_digit(img_path):
  img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
  gray = 255-img
  # применяем пороговую обработку
  (thresh, gray) = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

  # удаляем нулевые строки и столбцы
  while np.sum(gray[0]) == 0:
    gray = gray[1:]
  while np.sum(gray[:,0]) == 0:
    gray = np.delete(gray,0,1)
  while np.sum(gray[-1]) == 0:
    gray = gray[:-1]
  while np.sum(gray[:,-1]) == 0:
    gray = np.delete(gray,-1,1)
  rows,cols = gray.shape

  # изменяем размер, чтобы помещалось в box 20x20 пикселей
  if rows > cols:
    factor = 20.0/rows
    rows = 20
    cols = int(round(cols*factor))
    gray = cv2.resize(gray, (cols,rows))
  else:
    factor = 20.0/cols
    cols = 20
    rows = int(round(rows*factor))
    gray = cv2.resize(gray, (cols, rows))

  # расширяем до размера 28x28
  colsPadding = (int(math.ceil((28-cols)/2.0)),int(math.floor((28-cols)/2.0)))
  rowsPadding = (int(math.ceil((28-rows)/2.0)),int(math.floor((28-rows)/2.0)))
  gray = np.lib.pad(gray,(rowsPadding,colsPadding),'constant')

  # сдвигаем центр масс
  shiftx,shifty = getBestShift(gray)
  shifted = shift(gray,shiftx,shifty)
  gray = shifted

  cv2.imwrite('gray'+ img_path, gray)
  img = gray / 255.0
  img = np.array(img).reshape(-1, 28, 28, 1)
  out = str(np.argmax(model.predict(img)))
  return out

In [ ]:
print(rec_digit('doubleyou.jpg'))

***5 практика***


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input, decode_predictions


img_path = 'cat.jpg'  # Укажите путь к своему изображению
img = image.load_img(img_path, target_size=(32, 32))
img_array = image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)
img_array = preprocess_input(img_array)

predictions = model_7.predict(img_array)


# Вывод результатов
print(predictions)